In [1]:
import os

import numpy as np

import TwoPUtils
import STX3KO_analyses as stx



%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
def update_sess_dict(mouse, f, KO = True):
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
    if KO:
        try:
            source_folder  =os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus2','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
    else:
        try:
            source_folder  =os.path.join('/media','mplitt','Backup Plus3')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus5','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')

    f['mouse']=mouse
    f.update({'scan_file': source_stem + '.sbx',
              'scanheader_file': source_stem + '.mat',
              'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
              'scan_number': f['scan'],
              'prompt_for_keys': False,
              'VR_only': False,
              'scanner': "NLW",
             })
    return f

In [12]:

mouse = '4467333.1'
f = stx.ymaze_sess_deets.CTRL_sessions[mouse][8]
print(f)

({'date': '08_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2}, {'date': '08_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4})


In [13]:

if isinstance(f, dict):
    f = update_sess_dict(mouse,f,KO=False)
    scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
    if mouse in ['4467975.5', '4467333.1']:
        f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')
    else:
        f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')

    try:
        sess = stx.session.YMazeSession(**f)
        sess.load_scan_info()
        sess.align_VR_to_2P()
        sess.load_suite2p_data(which_ts=('F', 'Fneu'))
        sess.get_trial_info()
        sess.neuropil_corrected_dff()
        sess.place_cells_calc()
        TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')
    except:
        failed_sessions.append(f)

elif isinstance(f, tuple) or isinstance(f, list):
    total_frames = 0
    if mouse in ['4467975.5', '4467333.1']:
        s2p_path=os.path.join("/home/mplitt/2P_scratch",mouse,f[0]['date'],'combined','suite2p')
    else:
        s2p_path=os.path.join("/mnt/BigDisk/2P_scratch",mouse,f[0]['date'],'combined','suite2p')
    ops = np.load(os.path.join(s2p_path,"plane0","ops.npy"),allow_pickle=True).all()
    comb_frames = 0
    for _f in f:

        _f = update_sess_dict(mouse, _f,KO=False)
        if mouse in ['4467975.5', '4467333.1']:
            _f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')
        else:
            _f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')

       
        sess = stx.session.YMazeSession(**_f)
        sess.load_scan_info()
        sess.align_VR_to_2P()
        print(slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
        sess.load_suite2p_data(which_ts=('F', 'Fneu'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))

        sess.get_trial_info()
        sess.neuropil_corrected_dff()
        sess.place_cells_calc()
        TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')

        comb_frames += sess.scan_info['max_idx']

       
    print("ops max idx %d, combined sbx max idx %d" % (ops['nframes'], comb_frames))





/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:171: UserWarning: one frame correction
  warnings.warn('one frame correction')


Fixing teleports
(22933,) (8049,)
520.5797119757453 520.5376452753917
slice(0, 8048, None)
147 3752
4696 8047


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


...

159 3506
4440 7677
8614 12194
13131 19235


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0


/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:139: RuntimeWarning: Mean of empty slice
  SI = spatial_info(np.nanmean(C_trial_mat,axis=0),occ)
/home/mplitt/repos/TwoPUtils/TwoPUtils/spatial_analyses.py:150: RuntimeWarning: Mean of empty slice
  _SI_perm =  spatial_info(np.nanmean(C_trial_mat,axis=0),occ)


perm 0
ops max idx 27284, combined sbx max idx 27284


In [3]:
mouse = '4467331.1'
f = stx.ymaze_sess_deets.CTRL_sessions[mouse][6]
print(f)

({'date': '05_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}, {'date': '05_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4})


In [4]:
_f = update_sess_dict(mouse, f[0],KO=False)
if mouse in ['4467975.5', '4467333.1']:
    _f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')
else:
    _f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f[0]['mouse'],f[0]['date'],'combined','suite2p')


sess = stx.session.YMazeSession(**_f)
sess.load_scan_info()
sess.align_VR_to_2P()
# print(slice(comb_frames,comb_frames+sess.scan_info['max_idx']))
# sess.load_suite2p_data(which_ts=('F', 'Fneu'), frames = slice(comb_frames,comb_frames+sess.scan_info['max_idx']))

sess.get_trial_info()
# sess.neuropil_corrected_dff()
# sess.place_cells_calc()
# TwoPUtils.sess.save_session(sess,'/home/mplitt/YMazeSessPkls')


Fixing teleports
(25010,) (9340,)
604.1639717028802 604.0384032339566


/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:215: UserWarning: Number of teleports and trial starts don't match
  warnings.warn("Number of teleports and trial starts don't match")
/home/mplitt/repos/TwoPUtils/TwoPUtils/preprocessing.py:221: UserWarning: One more trial start than teleport, assuming the final teleport got chopped
  warnings.warn(('One more trial start than teleport, assuming the final teleport got chopped'))


In [5]:
print(sess.trial_start_inds)
print(sess.teleport_inds)

Int64Index([ 226,  470,  719, 1074, 1234, 1414, 1593, 1770, 1957, 2208, 2452,
            2641, 2843, 3093, 3306, 3559, 3753, 3918, 4137, 4342, 5421, 5613,
            5792, 5975, 6151, 6381, 6607, 6789, 7024, 7225, 7431, 7626, 7782,
            7944, 8159, 8343, 8524, 8851, 9002, 9210],
           dtype='int64')
Int64Index([ 397,  644, 1038, 1189, 1376, 1572, 1729, 1901, 2123, 2382, 2582,
            2788, 3007, 3277, 3474, 3721, 3901, 4086, 4315, 4482, 5591, 5751,
            5920, 6113, 6311, 6539, 6753, 6942, 7183, 7392, 7559, 7770, 7927,
            8112, 8306, 8470, 8826, 8949, 9133, 9339],
           dtype='int64')
